In [ ]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [ ]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

In [ ]:
CALLERS = ["A", "B", "C"]

@ray.remote
class MethodStateCounter:
    def __init__(self):
        self.invokers = {"A": [0], "B": [0], "C": [0]}
    
    def invoke(self, name):
        # pretend to do some work
        time.sleep(0.5)
        
        # compute a random result and update the result
        result = random.randint(5, 25)
        self.invokers[name].append(result)
        return result
        
    def get_invoker_state(self, name):
        return self.invokers[name]
    
    def get_number_of_calls(self, name):
        return len(self.invokers[name])
    
    def get_all_invoker_state(self):
        return self.invokers

In [ ]:
actor_instance = MethodStateCounter.remote()

In [ ]:
for _ in range(10):
    name = random.choice(CALLERS)
    actor_instance.invoke.remote(name)

#### 1. Get number of times an invoker name was called

In [ ]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_number_of_calls.remote(name))
print(f"Number of times: {name} was called: {count}")

#### 2. Get computed values by invoker name

In [ ]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_invoker_state.remote(name))
print(f"All values computed by: {name} is: {count}")

#### 3. Get state of all invokers

In [ ]:
print(f"State of all computed values for invokers: {ray.get(actor_instance.get_all_invoker_state.remote())}")

In [ ]:
ray.shutdown()